In [160]:
using PyCall

In [ ]:
include("../deps/build.jl")

## Getting client secrets etc
URL https://console.developers.google.com/apis/credentials/wizard?api=drive.googleapis.com
DOCS https://pythonhosted.org/PyDrive/quickstart.html#authentication

In [21]:
@pyimport pydrive.auth as pydrive_auth
function get_auth()
    if !isfile("client_secrets.json")
        println("Please go to https://console.developers.google.com/apis/credentials/wizard?api=drive.googleapis.com")
        println("and generate the credentials. for more info see https://pythonhosted.org/PyDrive/quickstart.html#authentication")
    end
    gauth = pydrive_auth.GoogleAuth()

    
    gauth[:LoadCredentialsFile]("creds.json")
    if gauth[:credentials] === nothing
        println("Please go to the following URL")
        println(gauth[:GetAuthUrl]())
        println("and paste the verification code")
        code = readline()
        gauth[:Auth](code)
    elseif gauth[:access_token_expired]
        gauth[:Refresh]()
    else
        # Initialize from saved creds
        gauth[:Authorize]()
    end
    # Save the current credentials to a file
    gauth[:SaveCredentialsFile]("creds.json")
    gauth
end

get_auth (generic function with 1 method)

In [42]:
@pyimport pydrive.drive as pydrive_drive
Drive(auth = get_auth()) = pywrap(pydrive_drive.GoogleDrive(auth))

drive=Drive()

__anon__

In [126]:
struct GDFile
    py::PyObject
end
Base.getindex(gd::GDFile, args...) = gd.py[args...]
metadata(gd::GDFile,field) = py"$(gd.py)[$field]"

function list_files(
        query = "'root' in parents and trashed=false",
        drive=Drive()
    )
    lf = pycall( drive.ListFile, PyObject, Dict("q"=>query))
    filelist = py"$lf.GetList()"o
    [GDFile(py"$filelist[$ii]"o) for ii in 0:py"len($filelist)"-1]
end


list_files (generic function with 3 methods)

In [128]:
is_google_mime(file)=startswith(metadata(file, "mimeType"), "application/vnd.google-apps")

is_google_mime (generic function with 1 method)

In [129]:
function list_files_in_folder(foldername, drive=Drive())
    folder = first(list_files("title='$foldername' and trashed=false", drive))
    @assert metadata(folder,"mimeType") == "application/vnd.google-apps.folder"
    folderid = metadata(folder, "id")
    list_files("'$(folderid)' in parents and trashed=false", drive)
end



list_files_in_folder (generic function with 3 methods)

LoadError: [91mMethodError: no method matching pywrap(::GDFile)[0m
Closest candidates are:
  pywrap([91m::PyCall.PyObject[39m, [91m::Symbol[39m) at /home/wheel/oxinabox/.julia/v0.6/PyCall/src/PyCall.jl:325
  pywrap([91m::PyCall.PyObject[39m) at /home/wheel/oxinabox/.julia/v0.6/PyCall/src/PyCall.jl:325[39m

In [151]:
function drive_download(remote::GDFile, localpath)
    remote[:GetContentFile](localpath)
    localpath
end


drive_download (generic function with 1 method)

In [142]:
using DataDeps

register(DataDep("GoogleDriveDemo",
        "Demonstration of google drive",
         list_files_in_folder("Demo"), #Issue: this actually runs the query at Start time, Replace with lazy vector
         fetch_method = drive_download));

In [143]:
Base.show(io::IO, gd::GDFile) = print(io, "Google Drive file: ", basename(gd))
Base.basename(gd::GDFile) = metadata(gd, "title")

In [154]:
collect(eachline(datadep"GoogleDriveDemo/text1.txt"))

INFO: This program has requested access to the data dependency GoogleDriveDemo.
INFO: which is not currently installed. It can be installed automatically, and you will not see this message again.
INFO: 
Demonstration of google drive

INFO: Do you want to download the dataset from GDFile[Google Drive file: text1.txt, Google Drive file: text2.txt, Google Drive file: image.bmp] to "/home/wheel/oxinabox/.julia/datadeps/GoogleDriveDemo"?
INFO: [y/n]
ERROR: MethodError: Cannot `convert` an object of type GDFile to an object of type HTTP.URIs.URI
This may have arisen from a call to the constructor HTTP.URIs.URI(...),
since type constructors fall back to convert methods.
ERROR: MethodError: Cannot `convert` an object of type GDFile to an object of type HTTP.URIs.URI
This may have arisen from a call to the constructor HTTP.URIs.URI(...),
since type constructors fall back to convert methods.
ERROR: MethodError: Cannot `convert` an object of type GDFile to an object of type HTTP.URIs.URI
This may

1-element Array{String,1}:
 "this is the content of the first text file"

In [156]:
readdir(datadep"GoogleDriveDemo")

3-element Array{String,1}:
 "image.bmp"
 "text1.txt"
 "text2.txt"

In [ ]:
rm(datadep"GoogleDriveDemo", recursive=true)